##### Import General Functions

In [49]:
from scipy.integrate import solve_ivp
import numpy as np
import pandas as pd

### Function to Run ODE - Deconstructing Fung et al., 2011

##### Import Functions and Dictionaries

In [10]:
def ODEfunction_DeconstructFung(
    NUMYEARS,
    time_points,  
    scenario,
    dictionary,
    function,
    xparam,
    init_cond):

    

    outputs = []
    
    for x in xparam:
    
        for C0 in np.linspace(0.01,0.99,2):

            
            #Initial Conditions-----------------------------------------------------
            C0, M0, T0 = C0, ((1-C0)/2), ((1-C0)/2)
            
            #Fung et al., 2011 CTMm ODE System-----------------------------------------------------
            lCs = dictionary['lCs'] 
            lCb = dictionary['lCb'] 
            εC  = dictionary['εC'] 
            rC  = dictionary['rC'] 
            αCT = dictionary['αCT'] 
            μC  = dictionary['μC']
            rM  = dictionary['rM'] 
            βM  = dictionary['βM'] 
            αMT = dictionary['αMT'] 
            αMC = dictionary['αMC']
            rT  = dictionary['rT'] 
            gT  = dictionary['gT'] 
            gM  = dictionary['gM'] 
            θ   = dictionary['θ']

            P = {
                'lCs': lCs, 
                'lCb': lCb, 
                'εC': εC, 
                'rC' : rC, 
                'αCT': αCT, 
                'μC' : μC, 
                'rM' : rM, 
                'βM': βM, 
                'αMT': αMT, 
                'αMC': αMC, 
                'rT' : rT, 
                'gT': gT, 
                'gM': gM, 
                'θ': θ}

             
            #Solve ODE-----------------------------------------------------
            sol = solve_ivp(function, [0, NUMYEARS], [C0, M0, T0], 
                            method = 'RK45', args = (P,), dense_output=True)

            N = sol.sol(time_points)

            C_array = N[0,:]
            M_array = N[1,:]
            T_array = N[2,:]

            
            #Ensure Run Reached Equilibrium-----------------------------------------------------
            thr_eq = 1e-06
            
            Cp = C_array[-1]
            Mp = M_array[-1]
            Tp = T_array[-1]
            
            Cp2 = C_array[-2]
            Mp2 = M_array[-2]
            Tp2 = T_array[-2]
           
            if abs(Cp - Cp2) < thr_eq and abs(Mp - Mp2) < thr_eq and abs(Tp - Tp2) < thr_eq:
                equil = 'yes'

            else:
                equil = 'no'


            #Categorize State at Equilibrium-----------------------------------------------------
            thr_ben = 0.01
            Fp = 1 - (Cp + Mp + Tp)
            
            C_t = Cp >= thr
            M_t = Mp >= thr
            T_t = Fp >= thr

            if C_t and not M_t and not T_t:
                ben_col = "allC"
                
            elif M_t and not C_t and not T_t:
                ben_col = "allM"
            
            elif T_t and not C_t and not M_t:
                ben_col = "allT"
            
            elif not C_t and not M_t and not T_t:
                ben_col = "allF"
            
            else:
                dom_val = max(Cp, Mp, Tp, Fp)
            
                if dom_val == Cp:
                    ben_col = "mostlyC" if T_t else "mostlyC_noT"
            
                elif dom_val == Mp:
                    ben_col = "mostlyM" if T_t else "mostlyM_noT"
            
                elif dom_val == Tp:
                    ben_col = "mostlyT"
                
                elif dom_val == Fp:
                    ben_col = "mostlyF"
                    
                else:
                    ben_col = "other"

                        
            #Save Results-----------------------------------------------------
            benthos = (rC, αCT, μC, rM, αMT, αMC, rT, gT, gM)
            state_eq = (Cp, Mp, Tp)
            labels = (ben_col, equil)

            
            outputs.append(
                benthos
                + state_eq
                + labels
             )

            
            benthos_c = ('rC', 'αCT', 'μC', 'rM', 'αMT', 'αMC', 'rT', 'gT', 'gM')
            state_eq_c = ('Cp', 'Mp', 'Tp')
            labels_c = ('ben_col', 'equil')

            columns = (
                benthos_c
                + state_eq_c
                + labels_c
            )

        
    df_prime = pd.DataFrame(outputs, columns=columns)
          
    return(df_prime)

### Function to Run ODE - Building Snyder ODE from Basal Fung et al., 2011 Model

##### Import Functions and Dictionaries

In [9]:
def ODEfunction_BuildSnyder(
    NUMYEARS,
    time_points,  
    init_cond,
    test_param_name,
    test_param_array,
    xparam_name,
    xparam_array,
    benthos_dictionary,
    herb_dictionary,
    function):

    #init_cond
    H0 = init_cond['H0']
    

    outputs = []

   
    for x in xparam_array:

        for test in test_param_array:
    
            for C0 in np.linspace(0.01,0.99,2):


                #Initial Conditions-----------------------------------------------------
                C0, M0, T0 = C0, ((1-C0)/2), ((1-C0)/2) 
  
                #Benthos Parameters-----------------------------------------------------
                rC  = benthos_dictionary['rC']
                αCT = benthos_dictionary['αCT']
                μC  = benthos_dictionary['μC']
                rM  = benthos_dictionary['rM']
                αMT = benthos_dictionary['αMT']
                αMC = benthos_dictionary['αMC']
                rT  = benthos_dictionary['rT']
                
                if xparam_name not in {'rM', 'μC'}:
                    raise ValueError(f"Invalid xparam_name: {xparam_name}")
                    
                if xparam_name   == 'rM':
                    rM = x
                elif xparam_name == 'μC':
                    μC = x
                
                #Herbivore Parameters-----------------------------------------------------
                s = herb_dictionary['s']
                h = herb_dictionary['h']
                η = herb_dictionary['η']
                e = herb_dictionary['e']
                μ = herb_dictionary['μ']
                θ = herb_dictionary['θ']
                γ = herb_dictionary['γ']
                φ = herb_dictionary['φ']

                if test_param_name not in {'h','η','e','μ','θ','γ','φ'}:
                    raise ValueError(f"Invalid test_param_name: {test_param_name}")
                    
                if test_param_name   == 'h':
                    h = test
                elif test_param_name == 'η':
                    η = test
                elif test_param_name == 'e':
                    e = test
                elif test_param_name == 'μ':
                    μ = test
                elif test_param_name == 'θ':
                    θ = test
                elif test_param_name == 'γ':
                    γ = test
                elif test_param_name == 'φ':
                    φ = test
                                
                #Solve ODE-----------------------------------------------------
                sol = solve_ivp(function, [0,NUMYEARS], [C0, M0, T0, H0], 
                                method = 'RK45', 
                                args = (rC, rM, rT, αCT, αMT, αMC, μC,                #benthos
                                        s, h, η, e, μ, θ, γ, φ),                      #general herbivore values
                                dense_output=True)

                N = sol.sol(time_points)

                C_array = N[0,:]
                M_array = N[1,:]
                T_array = N[2,:]
                
                H_array = N[3,:]

                
                #Ensure Run Reached Equilibrium-----------------------------------------------------
                thr_eq = 1e-06

                Cp = C_array[-1]
                Mp = M_array[-1]
                Tp = T_array[-1]

                Hp = H_array[-1]

                dC, dM, dT, dH = function(
                    time_points[-1],
                    [Cp, Mp, Tp, Hp],
                    rC, rM, rT, αCT, αMT, αMC, μC,
                    s, h, η, e, μ, θ, γ, φ
                )
                
                equil = (
                    abs(dC) < thr_eq and
                    abs(dM) < thr_eq and
                    abs(dT) < thr_eq and
                    abs(dH) < thr_eq
                )

                
                #Categorize State at Equilibrium-----------------------------------------------------
                thr_ben = 0.01

                #Benthos
                Fp = max(0.0, 1 - (Cp + Mp + Tp))
                C_t = Cp >= thr_ben
                M_t = Mp >= thr_ben
                T_t = Tp >= thr_ben

                if C_t and not M_t and not T_t:
                    ben_col = "allC"
                
                elif M_t and not C_t and not T_t:
                    ben_col = "allM"
                
                elif T_t and not C_t and not M_t:
                    ben_col = "allT"
                
                elif not C_t and not M_t and not T_t:
                    ben_col = "allF"
                
                else:
                    dom_val = max(Cp, Mp, Tp, Fp)
                
                    if dom_val == Cp:
                        ben_col = "mostlyC" if T_t else "mostlyC_noT"
                
                    elif dom_val == Mp:
                        ben_col = "mostlyM" if T_t else "mostlyM_noT"
                
                    elif dom_val == Tp:
                        ben_col = "mostlyT"
                    
                    elif dom_val == Fp:
                        ben_col = "mostlyF"
                        
                    else:
                        ben_col = "other"

        
                #Save Results-----------------------------------------------------
                benthos  = (rC, αCT, μC, rM, αMT, αMC, rT)
                herbivores = (s, h, η, e, μ, θ, γ, φ)
                state_eq = (Cp, Mp, Tp, Hp) #, Gp, Sp)
                labels   = (ben_col, equil) #, herb_col)
      
                                    
                outputs.append(
                    benthos
                    + herbivores
                    + state_eq
                    + labels
                 )
    
                
    benthos_c = ('rC', 'αCT', 'μC', 'rM', 'αMT', 'αMC', 'rT')
    herbivores_c = ('s', 'h', 'η', 'e', 'μ', 'θ', 'γ', 'φ')
    state_eq_c = ('Cp', 'Mp', 'Tp', 'Hp')
    labels_c = ('ben_col', 'equil')

    columns = (
        benthos_c
        + herbivores_c
        + state_eq_c
        + labels_c
    )

        
    df_prime = pd.DataFrame(outputs, columns=columns)
          
    return(df_prime)